In [ ]:
#Dowmload ollama
#ollama pull llama3 -- Downloading purpose
#ollama run llama3 -- Prompting
!pip install pyautogen[ollama]

In [ ]:
!pip install ollama fix-busted-json

In [1]:
import autogen
autogen.__version__

'0.10.2'

In [2]:
import agentops
agentops.init("<Enter your key>")

🖇 AgentOps: [ConcurrentFuturesInstrumentor] Successfully instrumented concurrent.futures.ThreadPoolExecutor
🖇 AgentOps: You're on the agentops free plan 🤔
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Sai Vennela Jagadesh\anaconda3\envs\SAI\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Sai Vennela Jagadesh\anaconda3\envs\SAI\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f914' in position 66: character maps to <undefined>
Call stack:
  File "C:\Users\Sai Vennela Jagadesh\anaconda3\envs\SAI\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Sai Vennela Jagadesh\anaconda3\envs\SAI\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  F

In [3]:
from pathlib import Path

from autogen import ConversableAgent
from autogen.coding import  CodeBlock, LocalCommandLineCodeExecutor
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

#Agent 1
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={
        "executor": executor,
    },
    human_input_mode="NEVER",
)

In [7]:
config_list = [
    {
        # Let's choose the Meta's Llama 3.1 model (model names must match Ollama exactly)
        "model": "llama3",
        # We specify the API Type as 'ollama' so it uses the Ollama client class
        "api_type": "ollama",
        "stream": False,
        #"client_host": "http://192.168.0.1:11434",
    }
]

In [8]:
# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, 
    print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the 
    task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your 
language skill.When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other 
action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. 
Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
"""

import os

#Agent 2

code_writer_agent = ConversableAgent(
    "code_writer",
    system_message=code_writer_system_message,
    llm_config={"config_list":config_list},
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",
)

In [9]:
import pprint

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent, message="""Write optimized Python code to calculate the number of unique permutations of the word 'ALGEBRA.'
    For example, the word 'ALGEbRA' should have 2520 unique permutations. Use efficient logic to compute the result and verify the output using 
    the word 'ALGEBRA.' The code should only return the final permutation count."""
)

pprint.pprint(chat_result)
#agentops.end_session("Success")

code_executor_agent (to code_writer):

Write optimized Python code to calculate the number of unique permutations of the word 'ALGEBRA.'
    For example, the word 'ALGEbRA' should have 2520 unique permutations. Use efficient logic to compute the result and verify the output using 
    the word 'ALGEBRA.' The code should only return the final permutation count.

--------------------------------------------------------------------------------
code_writer (to code_executor_agent):

# filename: calculate_permutations.py

import math

def calculate_permutations(word):
    # Calculate the number of unique permutations
    n = len(word)
    permutations_count = 1
    for char in set(word):
        # For each character, subtract one from the total count
        permutations_count *= math.factorial(n - word.count(char))
    return permutations_count

print(calculate_permutations('ALGEBRA'))

--------------------------------------------------------------------------------
code_executor_agent (to

In [10]:
chat_result.chat_history

[{'content': "Write optimized Python code to calculate the number of unique permutations of the word 'ALGEBRA.'\n    For example, the word 'ALGEbRA' should have 2520 unique permutations. Use efficient logic to compute the result and verify the output using \n    the word 'ALGEBRA.' The code should only return the final permutation count.",
  'role': 'assistant',
  'name': 'code_executor_agent'},
 {'content': "# filename: calculate_permutations.py\n\nimport math\n\ndef calculate_permutations(word):\n    # Calculate the number of unique permutations\n    n = len(word)\n    permutations_count = 1\n    for char in set(word):\n        # For each character, subtract one from the total count\n        permutations_count *= math.factorial(n - word.count(char))\n    return permutations_count\n\nprint(calculate_permutations('ALGEBRA'))",
  'role': 'user',
  'name': 'code_writer'},
 {'content': '', 'role': 'assistant', 'name': 'code_executor_agent'},
 {'content': "When you run this Python script w

In [11]:
chat_result.chat_id

10991230698298627381228358013870041902

In [12]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 0.0,
  'llama3': {'cost': 0.0,
   'prompt_tokens': 969,
   'completion_tokens': 309,
   'total_tokens': 1278}},
 'usage_excluding_cached_inference': {'total_cost': 0.0,
  'llama3': {'cost': 0.0,
   'prompt_tokens': 969,
   'completion_tokens': 309,
   'total_tokens': 1278}}}

In [13]:
chat_result.human_input

[]

In [14]:
chat_result.summary

''